In [ ]:
!nvidia-smi

Mon Apr  8 14:11:09 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.06              Driver Version: 545.23.06    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|


|   0  NVIDIA GeForce RTX 4090        On  | 00000000:C2:00.0 Off |                  Off |
| 30%   27C    P8              31W / 450W |      3MiB / 24564MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+----------------------+
                                                                                         
+---------------------------------------------------------------------------------------+
| Processes:                                                                            |
|  GPU   GI   CI        PID   Type   Process name                            GPU Memory |
|        ID   ID                                                             Usage      |
|=======================================================================================|
|  No running processes found                                                           |
+---------

In [ ]:
from src.segmentation.framework_handlers import yolo_handler
yolov8_handler.validate_version_and_gpu()

Ultralytics YOLOv8.0.229 🚀 Python-3.10.13 torch-2.1.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4090, 24217MiB)
Setup complete ✅ (32 CPUs, 251.7 GB RAM, 31.8/332.3 GB disk)


### Prepare Dataset
In this section, the dataset along with YOLO annotations is processed to conform to the input structure required by YOLO. The YOLO format necessitates organizing the dataset into two main directories, `images` and `labels`. Each of these directories is further divided into two subdirectories, `train` and `val`, for training and validation datasets respectively. This structure facilitates YOLO's training and validation processes.

In [ ]:
# prepare data to fine tune yolo v8
from src.segmentation.framework_handlers.yolo_handler import prepare_data_for_yolo
annotations_folder_name = "etaylor_cannabis_patches_test"
dataset_version = "v0.1"
saving_yaml_path = "/home/etaylor/code_projects/thesis/src/segmentation/notebooks/YOLO8"
yaml_file_path = prepare_data_for_yolo(annotations_folder_name, dataset_version, saving_yaml_path, train_percentage=0.8)

Annotations /sise/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test/annotations/export_coco-instance_etaylor_cannabis_patches_test_v0.1.json:  53%|█████▎    | 47/89 [00:00<00:00, 465.70it/s]

Annotations /sise/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test/annotations/export_coco-instance_etaylor_cannabis_patches_test_v0.1.json: 100%|██████████| 89/89 [00:00<00:00, 479.47it/s]

COCO data converted successfully.
Results saved to /sise/home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test/annotations/yolo


The folder /home/etaylor/code_projects/thesis/segments/etaylor_cannabis_patches_test/annotations/yolo has been deleted.


### Ultralytics Settings

In here we can modify the settings of the yolo runs. 

In [ ]:
from ultralytics import settings

runs_dir = "/home/etaylor/code_projects/thesis/src/segmentation/notebooks/YOLO8/runs"
# Update a setting
settings.update({'runs_dir': runs_dir})

# View all settings
print(settings)

{'settings_version': '0.0.4', 'datasets_dir': '/sise/home/etaylor/datasets', 'weights_dir': '/home/etaylor/weights', 'runs_dir': '/home/etaylor/code_projects/thesis/src/segmentation/notebooks/YOLO8/runs', 'uuid': '958bdde9742b290d62523897603d59c1039672389661ad1a173c1e862ebe77c6', 'sync': True, 'api_key': '', 'clearml': True, 'comet': True, 'dvc': True, 'hub': True, 'mlflow': True, 'neptune': True, 'raytune': True, 'tensorboard': False, 'wandb': True}


In [7]:
# in case dataset already converted from coco to yolo format use manual path
yaml_path = "/home/etaylor/code_projects/thesis/src/segmentation/notebooks/YOLO8/instance_segmentation/data.yaml"

### Find best Hyper params with [Ray Tune](https://docs.ultralytics.com/integrations/ray-tune/#custom-search-space-example)

In the provided code snippet, Ray Tune is utilized for hyperparameter tuning of a YOLOv8n segmentation model using a specific model checkpoint and dataset defined in a YAML file.

In [ ]:
from ultralytics import YOLO

model_checkpoint_path = '/home/etaylor/code_projects/thesis/checkpoints/yolo/yolov8n-seg.pt'
model = YOLO(model_checkpoint_path) # pass any model type

# Start tuning hyperparameters for YOLOv8n training on the COCO8 dataset
result_grid = model.tune(data=yaml_file_path, use_ray=True)

### Basic Trial-Level Analysis
Access individual trial hyperparameter configurations and the last reported metrics.



In [10]:
for i, result in enumerate(result_grid):
    print(f"Trial #{i}: Configuration: {result.config}, Last Reported Metrics: {result.metrics}")

Trial #0: Configuration: {'lr0': 0.013022099726378083, 'lrf': 0.9437005496597051, 'momentum': 0.6975878806580793, 'weight_decay': 0.0008342974436230575, 'warmup_epochs': 0.25155517091431046, 'warmup_momentum': 0.13049383735872452, 'box': 0.12091941281814352, 'cls': 1.1190074351950952, 'hsv_h': 0.01450226465606047, 'hsv_s': 0.22940964929390675, 'hsv_v': 0.35213392324608483, 'degrees': 24.03759343592748, 'translate': 0.07232507054782537, 'scale': 0.4843422241740211, 'shear': 5.19509841780944, 'perspective': 0.0006856305105964295, 'flipud': 0.18766660600441276, 'fliplr': 0.17761833344872546, 'mosaic': 0.4338229868148046, 'mixup': 0.9853802344738387, 'copy_paste': 0.827117390211561, 'data': '/home/etaylor/code_projects/thesis/src/segmentation/notebooks/YOLO8/instance_segmentation/data.yaml'}, Last Reported Metrics: {'trial_id': '2c333_00000', 'date': '2024-04-06_18-41-50', 'timestamp': 1712418110, 'pid': 16218, 'hostname': 'cs-pheno-24', 'node_ip': '132.72.40.55', 'config': {'lr0': 0.01302

#### Plotting the Entire History of Reported Metrics for a Trial


In [ ]:
import matplotlib.pyplot as plt

for result in result_grid:
    plt.plot(result.metrics_dataframe["training_iteration"], result.metrics_dataframe["mean_accuracy"], label=f"Trial {i}")

plt.xlabel('Training Iterations')
plt.ylabel('Mean Accuracy')
plt.legend()
plt.show()